In [1]:
# import all necessary libraries
import os
from datetime import datetime, timedelta
import getpass
import xarray as xr
!python -m pip install motuclient==1.8.4 --no-cache-dir
import motuclient
import xarray as xr
import numpy as np
!python -m pip install ffmpeg==1.4
import ffmpeg
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import pandas as pd
!python -m pip install cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import dotenv
dotenv.load_dotenv("dev.env")

  Preparing metadata (setup.py) ... done
  Created wheel for motuclient: filename=motuclient-1.8.4-py3-none-any.whl size=32873 sha256=fc0f6f47aac6f790b58b39c882ec856ef2cc9284cd27d740ac16089b80d30066
  Stored in directory: /tmp/pip-ephem-wheel-cache-6c7xl7p8/wheels/46/74/d1/ee3216f52d83212f10a0a5f5b94f3d99cf3d29b8ecf09c60b6
Successfully built motuclient
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=87b305b2d79b1ab1e8c2b11214d0f4f4dfea770e163af24450d2ae36d0009a11
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 70.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.9 MB/s eta 0:00:00
  Created wheel for cartopy:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# this is the directory that will be created
#path = r'/content/drive/MyDrive/MasterThesis/Data/CHL_300m'
path = r'/content/drive/MyDrive/MasterThesis/Predictions/Data'

#use os.mkdir() to create the directory
isExist = os.path.exists(path)
if not isExist:
    os.mkdir(path)

# Copernicus Marine Credentials
# load credentials from environment

USERNAME = os.getenv("CMEMS_USERNAME")
PASSWORD = os.getenv("CMEMS_PASSWORD")

# Work directory
out_dir = path

# product and dataset IDs
serviceID = "OCEANCOLOUR_BAL_BGC_L3_NRT_009_131" # OCEANCOLOUR_BAL_BGC_L3_NRT_009_131
productID = "cmems_obs-oc_bal_bgc-plankton_nrt_l3-olci-300m_P1D"    # cmems_obs-oc_bal_bgc-plankton_nrt_l3-olci-300m_P1D

#coordinates
lon = (9.25 , 30.25) # 9.25 to 30.25
lat = (53.25 , 65.85) #  53.25 to 65.85

# variable
var = "--variable CHL"

# Boundary dates
start_date = datetime(2023, 6, 20, 0)
end_date = datetime(2023, 6, 29, 23)

print(start_date, end_date)

# delta time
delta_t = start_date-end_date   # d+daydays to download by request
print(delta_t)
delta_t = timedelta(days=22)
print(start_date+delta_t)

2023-06-20 00:00:00 2023-06-29 23:00:00
-10 days, 1:00:00
2023-07-12 00:00:00


In [4]:
# Download loop
while start_date <= end_date:
    temp_endday = start_date + delta_t
    # Output filename
    out_name = f"CHL_data_{start_date.day}{start_date.month}{start_date.year}_to_{temp_endday.day}{temp_endday.month}{temp_endday.year}.nc"

    # Motuclient command line
    query = f'python -m motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu \
    --service-id {serviceID}-TDS --product-id {productID} \
    --longitude-min {lon[0]} --longitude-max {lon[1]} --latitude-min {lat[0]} --latitude-max {lat[1]}\
    --date-min "{start_date}" --date-max "{temp_endday}" \
    {var} \
    --out-dir {out_dir} --out-name {out_name} --user {USERNAME} --pwd {PASSWORD}'

    print(f"============== Running request on {start_date} ==============")

    result = subprocess.run(query, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = result.stdout.decode('utf-8')
    error = result.stderr.decode('utf-8')

    print(output)
    print(error)

    start_date=temp_endday

print(f"============== Download completed! All files are in your directory {out_dir} ==============")

============== Running request on 2023-06-20 00:00:00 ==============
2023-06-30 05:28:49.849 [ INFO] Asynchronous mode set
2023-06-30 05:28:49.850 [ INFO] Authenticating user noehlckers for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-30 05:28:54.146 [ INFO] Requesting file to download (this can take a while)...
2023-06-30 05:29:01.275 [ INFO] Authenticating user noehlckers for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-30 05:29:11.793 [ INFO] Product is not yet available (request in progress)
2023-06-30 05:29:21.800 [ INFO] Authenticating user noehlckers for service https://nrt.cmems-du.eu/motu-web/Motu
2023-06-30 05:29:32.770 [ INFO] The product is ready for download
2023-06-30 05:29:32.770 [ INFO] Downloading file (this can take a while)...
2023-06-30 05:29:33.428 [ INFO] File type: application/x-netcdf
2023-06-30 05:29:33.428 [ INFO] File size: 590.8 MB (590787316 B)
2023-06-30 05:29:33.428 [ INFO] Downloading file /content/drive/MyDrive/MasterThesis/Predictions

In [5]:
# Merge data
print(path)
print(out_name)
#out_name = "CHL_data_762023_to_2462023.nc"
merged_nc = xr.open_dataset(f"{path}/{out_name}")

/content/drive/MyDrive/MasterThesis/Predictions/Data
CHL_data_2062023_to_1272023.nc


In [ ]:
merged_nc

<xarray.Dataset>
Dimensions:  (time: 9, lat: 4676, lon: 3948)
Coordinates:
  * time     (time) datetime64[ns] 2023-06-04 2023-06-18 ... 2023-06-25
  * lat      (lat) float32 53.25 53.25 53.26 53.26 ... 65.84 65.84 65.85 65.85
  * lon      (lon) float32 9.253 9.258 9.263 9.269 ... 30.23 30.24 30.24 30.25
Data variables:
    CHL      (time, lat, lon) float32 ...
Attributes: (12/44)
    netcdf_version:          v4
    Conventions:             CF-1.4
    platform:                Sentinel3A+B
    product_level:           L3
    contact:                 technical@gos.artov.isac.cnr.it
    institution:             CNR-GOS
    ...                      ...
    cmems_product_id:        OCEANCOLOUR_BAL_BGC_L3_NRT_009_131
    History:                 Translated to CF-1.0 Conventions by Netcdf-Java ...
    geospatial_lat_min:      53.251346588134766
    geospatial_lat_max:      65.8486557006836
    geospatial_lon_min:      9.252659797668457
    geospatial_lon_max:      30.24734115600586